In [5]:
import argparse

import os
import sys
import glob
import time
import random

import numpy as np
#from scipy.misc import imread, imresize, imsave
from skimage.io import imread, imsave
from skimage.transform import resize as imresize
from matplotlib import pyplot as plt

from utils.rgb_ind_convertor import *
from utils.util import *

In [12]:
def post_process(input_dir, save_dir, merge=True):
	if not os.path.exists(save_dir):
		os.mkdir(save_dir)

	input_paths = sorted(glob.glob(os.path.join(input_dir, '*.png')))
	inputs = []
	for i in input_paths:
		if not i.endswith('image.png'):
			inputs.append(i)
	names = [i.split('/')[-1] for i in input_paths]
	out_paths = [os.path.join(save_dir, i) for i in names]

	n = len(inputs)
	# n = 1
	for i in range(n):
		im = imread(inputs[i], pilmode='RGB')
		im_ind = rgb2ind(im, color_map=floorplan_fuse_map)
		# seperate image into room-seg & boundary-seg
		rm_ind = im_ind.copy()
		rm_ind[im_ind==9] = 0
		rm_ind[im_ind==10] = 0

		bd_ind = np.zeros(im_ind.shape, dtype=np.uint8)
		bd_ind[im_ind==9] = 9
		bd_ind[im_ind==10] = 10

		hard_c = (bd_ind>0).astype(np.uint8)

		# region from room prediction it self
		rm_mask = np.zeros(rm_ind.shape)
		rm_mask[rm_ind>0] = 1			
		# region from close_wall line		
		cw_mask = hard_c
		# refine close wall mask by filling the grap between bright line	
		cw_mask = fill_break_line(cw_mask)
			
		fuse_mask = cw_mask + rm_mask
		fuse_mask[fuse_mask>=1] = 255

		# refine fuse mask by filling the hole
		fuse_mask = flood_fill(fuse_mask)
		fuse_mask = fuse_mask // 255	

		# one room one label
		new_rm_ind = refine_room_region(cw_mask, rm_ind)

		# ignore the background mislabeling
		new_rm_ind = fuse_mask*new_rm_ind

		print('Saving {}th refined room prediciton to {}'.format(i, out_paths[i]))
		if merge:
			new_rm_ind[bd_ind==9] = 9
			new_rm_ind[bd_ind==10] = 10
			rgb = ind2rgb(new_rm_ind, color_map=floorplan_fuse_map)
		else:
			rgb = ind2rgb(new_rm_ind)
		imsave(out_paths[i], rgb)



In [13]:
input_dir = '/d2/studies/TF2DeepFloorplan/outJan5/'
save_dir = '/d2/studies/TF2DeepFloorplan/dataset/results/'

In [14]:
post_process(input_dir, save_dir, merge=True)

Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
/home/smith/anaconda3/envs/dfp/lib/python3.7/site-packages/ipykernel_launcher.py:57: UserWarning: /d2/studies/TF2DeepFloorplan/dataset/results/0_bounds.png is a low contrast image
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
/home/smith/anaconda3/envs/dfp/lib/python3.7/site-packages/ipykernel_launcher.py:57: UserWarning: /d2/studies/TF2DeepFloorplan/dataset/results/0_image.png is a low contrast image
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
/home/smith/anaconda3/envs/dfp/lib/python3.7/site-packages/ipykernel_launcher.py:57: UserWarning: /d2/studies/TF2DeepFloorplan/dataset/results/0_logcw.png is a low contrast image
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to sup

Saving 0th refined room prediciton to /d2/studies/TF2DeepFloorplan/dataset/results/0.png
Saving 1th refined room prediciton to /d2/studies/TF2DeepFloorplan/dataset/results/0_bounds.png
Saving 2th refined room prediciton to /d2/studies/TF2DeepFloorplan/dataset/results/0_image.png
Saving 3th refined room prediciton to /d2/studies/TF2DeepFloorplan/dataset/results/0_logcw.png
Saving 4th refined room prediciton to /d2/studies/TF2DeepFloorplan/dataset/results/0_logr.png
Saving 5th refined room prediciton to /d2/studies/TF2DeepFloorplan/dataset/results/0_rooms.png


/home/smith/anaconda3/envs/dfp/lib/python3.7/site-packages/ipykernel_launcher.py:57: UserWarning: /d2/studies/TF2DeepFloorplan/dataset/results/10.png is a low contrast image
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
/home/smith/anaconda3/envs/dfp/lib/python3.7/site-packages/ipykernel_launcher.py:57: UserWarning: /d2/studies/TF2DeepFloorplan/dataset/results/10_bounds.png is a low contrast image
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
/home/smith/anaconda3/envs/dfp/lib/python3.7/site-packages/ipykernel_launcher.py:57: UserWarning: /d2/studies/TF2DeepFloorplan/dataset/results/10_image.png is a low contrast image
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
/home/smith/anaconda3/envs/dfp/lib/python3.7/site-packages/ipykernel_launcher.py:57: UserWarning: /d2/studie

Saving 6th refined room prediciton to /d2/studies/TF2DeepFloorplan/dataset/results/10.png
Saving 7th refined room prediciton to /d2/studies/TF2DeepFloorplan/dataset/results/10_bounds.png
Saving 8th refined room prediciton to /d2/studies/TF2DeepFloorplan/dataset/results/10_image.png
Saving 9th refined room prediciton to /d2/studies/TF2DeepFloorplan/dataset/results/10_logcw.png
Saving 10th refined room prediciton to /d2/studies/TF2DeepFloorplan/dataset/results/10_logr.png
Saving 11th refined room prediciton to /d2/studies/TF2DeepFloorplan/dataset/results/10_rooms.png
Saving 12th refined room prediciton to /d2/studies/TF2DeepFloorplan/dataset/results/20.png


/home/smith/anaconda3/envs/dfp/lib/python3.7/site-packages/ipykernel_launcher.py:57: UserWarning: /d2/studies/TF2DeepFloorplan/dataset/results/20_bounds.png is a low contrast image
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
/home/smith/anaconda3/envs/dfp/lib/python3.7/site-packages/ipykernel_launcher.py:57: UserWarning: /d2/studies/TF2DeepFloorplan/dataset/results/20_image.png is a low contrast image
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
/home/smith/anaconda3/envs/dfp/lib/python3.7/site-packages/ipykernel_launcher.py:57: UserWarning: /d2/studies/TF2DeepFloorplan/dataset/results/20_logr.png is a low contrast image
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to s

...

Saving 27th refined room prediciton to /d2/studies/TF2DeepFloorplan/dataset/results/40_logcw.png
Saving 28th refined room prediciton to /d2/studies/TF2DeepFloorplan/dataset/results/40_logr.png
Saving 29th refined room prediciton to /d2/studies/TF2DeepFloorplan/dataset/results/40_rooms.png
Saving 30th refined room prediciton to /d2/studies/TF2DeepFloorplan/dataset/results/50.png
Saving 31th refined room prediciton to /d2/studies/TF2DeepFloorplan/dataset/results/50_bounds.png
Saving 32th refined room prediciton to /d2/studies/TF2DeepFloorplan/dataset/results/50_image.png
Saving 33th refined room prediciton to /d2/studies/TF2DeepFloorplan/dataset/results/50_logcw.png
Saving 34th refined room prediciton to /d2/studies/TF2DeepFloorplan/dataset/results/50_logr.png


/home/smith/anaconda3/envs/dfp/lib/python3.7/site-packages/ipykernel_launcher.py:57: UserWarning: /d2/studies/TF2DeepFloorplan/dataset/results/50_logr.png is a low contrast image
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
/home/smith/anaconda3/envs/dfp/lib/python3.7/site-packages/ipykernel_launcher.py:57: UserWarning: /d2/studies/TF2DeepFloorplan/dataset/results/60.png is a low contrast image
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
/home/smith/anaconda3/envs/dfp/lib/python3.7/site-packages/ipykernel_launcher.py:57: UserWarning: /d2/studies/TF2DeepFloorplan/dataset/results/60_bounds.png is a low contrast image
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppres

Saving 35th refined room prediciton to /d2/studies/TF2DeepFloorplan/dataset/results/50_rooms.png
Saving 36th refined room prediciton to /d2/studies/TF2DeepFloorplan/dataset/results/60.png
Saving 37th refined room prediciton to /d2/studies/TF2DeepFloorplan/dataset/results/60_bounds.png
Saving 38th refined room prediciton to /d2/studies/TF2DeepFloorplan/dataset/results/60_image.png
Saving 39th refined room prediciton to /d2/studies/TF2DeepFloorplan/dataset/results/60_logcw.png
Saving 40th refined room prediciton to /d2/studies/TF2DeepFloorplan/dataset/results/60_logr.png
Saving 41th refined room prediciton to /d2/studies/TF2DeepFloorplan/dataset/results/60_rooms.png


/home/smith/anaconda3/envs/dfp/lib/python3.7/site-packages/ipykernel_launcher.py:57: UserWarning: /d2/studies/TF2DeepFloorplan/dataset/results/70.png is a low contrast image
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
/home/smith/anaconda3/envs/dfp/lib/python3.7/site-packages/ipykernel_launcher.py:57: UserWarning: /d2/studies/TF2DeepFloorplan/dataset/results/70_bounds.png is a low contrast image
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
/home/smith/anaconda3/envs/dfp/lib/python3.7/site-packages/ipykernel_launcher.py:57: UserWarning: /d2/studies/TF2DeepFloorplan/dataset/results/70_image.png is a low contrast image
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


Saving 42th refined room prediciton to /d2/studies/TF2DeepFloorplan/dataset/results/70.png
Saving 43th refined room prediciton to /d2/studies/TF2DeepFloorplan/dataset/results/70_bounds.png
Saving 44th refined room prediciton to /d2/studies/TF2DeepFloorplan/dataset/results/70_image.png
